## Sandbox for interacting with the ARASAAC API

In [2]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

ModuleNotFoundError: No module named 'pandas'

In [ ]:
base_url = "https://api.arasaac.org/api"

def get_all_pictographs(language="en"):
    url = f"{base_url}/pictograms/{language}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None